In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os    
    

<h3>basic block 정의 (residual block)</h3>

In [3]:
class BasicBlock(nn.Module):  ## 파이토치의 nn.module을 상속받아 BasicBlock을 생성
    def __init__(self, in_planes, out_planes, stride=1): #BasicBlock 클래스의 초기화 매서드입니다. in_planes(입력 필터 갯수), out_planes(출력 필터 갯수)
        super(BasicBlock, self).__init__() # nn.Module의 초기화 메서드를 호출합니다.
        
        # nn.Conv2d 를 사용하여 2d 컨볼루션 레이어 conv1을 생성함, 입력 채널(in_planes), 출력채널 (out_planes), 커널 크기 3x3, 스트라이드 1, 편향 비활성화를 설정
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias = False)
        # nn.BatchNorm2d 를 사용하여 배치 정규화 과정을 수행한다. 입력채널수는 outplanes의 값을 받음
        self.bn1 = nn.BatchNorm2d(out_planes)
        
    
        # 두번째 컨볼루션 레이어를 생성하며 설정은 이전 레이어와 동일
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        
        # shortcut 연결을 위한 빈 Sequential 객체 생성 및 self.shortcut에 인스턴스에 할당 stride 값이 1인 경우 해당하는 부분에서 shortcut 연결이 필요하지 않기 때문
        self.shortcut = nn.Sequential()
        
        # 입력 데이터의 stride 값이 1이 아닌 경우에만 shortcut 연결을 수행함, 1인 경우에는 입력과 출력의 크기가 동일하므로 추가적인 처리가 필요하지 않음
        if stride != 1:
            # shortcut 연산을 위해 Sequential 객체를 새롭게 정의한다 stride 값이 1이 아닌경우, 이 객체는 1x1 컨볼루션 연산과 batch normalization을 수행함
            self.shortcut = nn.Sequential(
            nn.Conv2d(in_planes, out_planes, kernel_size= 1, stride=stride, bias=False),
            nn.BatchNorm2d(out_planes)
        )
            
    # forward(순전파) 주어진 입력 x를 기반으로 신경망의 순전파 연산을 수행함
        
    def forward(self, x):
        out = F.relu(self.bnl(self.conv1(x))) # 컨볼루션 연산을 통해 처리한 결과(self.conv1) 'x' 를 self.bnl과 relu 활성화 함수를 거쳐 out 변수에 저장
        out = self.bn2(self.conv2(out)) # 이전 단계의 결과인 out을 self.conv2 연산을 통해 다시 처리하고, 그결과를 배치 정규화를 거쳐 다시 out 변수에 저장함
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out) # skip connection을 거쳐 최종적으로 합 연산된 out을 relu를 통해 activation
        return(out)

<h3>모델 구현</h3>

In [10]:
class ResNet(nn.Module): # ResNet 클래스 정의
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        
        self.in_planes = 64
        
        #64개의 3x3 필터를 사용
        self.conv1 = nn.Conv2d(3,64, kernel_size=3, stride=1, padding=1, bias = False)
         
        self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self.make_layer(block, 64, num_blocks[2], stride=2)
        self.layer4 = self.make_layer(block, 64, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)
    
    def _make_layer(block, outplanes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, outplanes, stride))
            self.in_planes = outplanes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return(out)

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])